# 🏪 Minimart POS Database Design & Core Queries (Group Project Deliverable)
## 1. Business Domain and Key Activities
**Domain:** Minimart Point-of-Sale (POS) and Inventory Management System.
**Objective:** To design and implement a system for cashiers to process sales and managers to handle inventory, user, and promotion management.

This notebook contains the complete SQLModel schema definitions and the core Python logic snippets (FastAPI/SQLModel) for 14 critical database interactions in the Minimart POS backend.

| Activity/Process           | Description                                                                                       | Corresponding Backend File                                   |
|---------------------------|---------------------------------------------------------------------------------------------------|--------------------------------------------------------------|
| **Sales Transaction/Checkout** | Processing a sale, applying product/member discounts, and finalizing payment.                  | `backend/app/routes/transactions.py`                         |
| **Inventory Management**       | Creating new products, modifying product details (e.g., price, promotion ID), adjusting stock. | `backend/app/routes/products.py`                             |
| **Promotion Management**       | Creating, updating, and deleting time-based or percentage-based product promotions.            | `backend/app/routes/promotions.py`                           |
| **Sales/Employee Reporting**   | Managers viewing transaction history, KPIs, and filtering employees.                          | `backend/app/routes/users.py`, `backend/app/routes/transactions.py` |
| **Member Loyalty Update**      | Updating member points, total spend, and tier after each transaction.                         | `backend/app/routes/transactions.py`                         |


## 1. Database Schema Definitions (Full Code)

In [ ]:
# ================================================================
# Full SQLModel Definitions for solid-fishstick Backend
# ================================================================
from typing import Optional
from decimal import Decimal
from datetime import datetime, date
from uuid import uuid4

from sqlmodel import SQLModel, Field, Column
from sqlalchemy.types import Numeric
from sqlalchemy import CheckConstraint
from passlib.context import CryptContext
import sqlalchemy as sa


pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

# ================================================================
# User Model
# ================================================================
class User(SQLModel, table=True):
    __tablename__ = "user"

    uid: str = Field(
        default_factory=lambda: str(uuid4())[:10],
        primary_key=True,
        index=True
    )
    email: str = Field(
        index=True,
        sa_column_kwargs={"unique": True}
    )
    username: str = Field(
        index=True,
        sa_column_kwargs={"unique": True}
    )
    name: str
    hashed_password: str
    role: str = Field(default="cashier")
    is_active: bool = Field(default=True)
    created_at: datetime = Field(default_factory=datetime.utcnow)


# ================================================================
# Cashier Model
# ================================================================
class Cashier(SQLModel, table=True):
    employee_id: str = Field(
        primary_key=True,
        foreign_key="user.uid"
    )
    position: Optional[str] = None


# ================================================================
# Manager Model
# ================================================================
class Manager(SQLModel, table=True):
    admin_id: str = Field(
        primary_key=True,
        foreign_key="user.uid"
    )


# ================================================================
# Membership Tier Model
# ================================================================
class MembershipTier(SQLModel, table=True):
    __tablename__ = "membershiptier"

    rank_name: str = Field(primary_key=True)
    min_spent: Decimal = Field(sa_column=Column(Numeric(8, 2)))
    max_spent: Optional[Decimal] = Field(
        default=None,
        sa_column=Column(Numeric(12, 2))
    )
    discount_rate: Decimal = Field(sa_column=Column(Numeric(5, 2)))
    benefits: Optional[str] = None


# ================================================================
# Member Model
# ================================================================
class Member(SQLModel, table=True):
    member_id: str = Field(primary_key=True)
    phone: str = Field(index=True, sa_column_kwargs={"unique": True})
    points_balance: int = Field(default=0)
    total_spent: Decimal = Field(
        default=Decimal("0.00"),
        sa_column=Column(Numeric(12, 2))
    )
    membership_rank: str = Field(
        default="Bronze",
        foreign_key="membershiptier.rank_name"
    )
    discount_rate: Decimal = Field(
        default=Decimal("3.00"),
        sa_column=Column(Numeric(5, 2))
    )
    registration_date: date

    __table_args__ = (
        CheckConstraint("points_balance >= 0"),
        CheckConstraint("total_spent >= 0"),
    )


# ================================================================
# Promotion Model
# ================================================================
class Promotion(SQLModel, table=True):
    promotion_id: Optional[int] = Field(default=None, primary_key=True)
    promotion_name: str
    discount_type: str
    discount_value: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    start_date: date
    end_date: date
    is_active: bool = Field(default=True)

    __table_args__ = (
        CheckConstraint("discount_type IN ('PERCENTAGE', 'FIXED')"),
        CheckConstraint("discount_value > 0"),
        CheckConstraint("end_date >= start_date"),
    )


# ================================================================
# Product Model
# ================================================================
class Product(SQLModel, table=True):
    product_id: Optional[int] = Field(default=None, primary_key=True)
    barcode: str = Field(sa_column_kwargs={"unique": True})
    name: str
    brand: str
    category: str
    cost_price: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    selling_price: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    stock_quantity: int = Field(default=0)
    min_stock: int = Field(default=10)
    promotion_id: Optional[int] = Field(default=None, foreign_key="promotion.promotion_id")

    __table_args__ = (
        CheckConstraint("selling_price >= cost_price"),
        CheckConstraint("stock_quantity >= 0"),
        CheckConstraint("min_stock > 0"),
    )


# ================================================================
# Transaction Model
# ================================================================
class Transaction(SQLModel, table=True):
    __tablename__ = "transaction"

    transaction_id: Optional[int] = Field(default=None, primary_key=True)
    transaction_date: datetime = Field(default_factory=datetime.utcnow)
    employee_id: str = Field(foreign_key="cashier.employee_id")
    member_id: Optional[str] = Field(default=None, foreign_key="member.member_id")

    subtotal: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    product_discount: Decimal = Field(default=Decimal("0.00"), sa_column=Column(Numeric(10, 2)))
    membership_discount: Decimal = Field(default=Decimal("0.00"), sa_column=Column(Numeric(10, 2)))
    total_amount: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    payment_method: str

    __table_args__ = (
        CheckConstraint("payment_method IN ('Cash','Card','QR Code')"),
        CheckConstraint("total_amount = subtotal - membership_discount"),
    )


# ================================================================
# Transaction Item Model
# ================================================================
class TransactionItem(SQLModel, table=True):
    transaction_id: int = Field(foreign_key="transaction.transaction_id", primary_key=True)
    product_id: int = Field(foreign_key="product.product_id", primary_key=True)

    quantity: int
    unit_price: Decimal = Field(sa_column=Column(Numeric(10, 2)))
    discount_amount: Decimal = Field(default=Decimal("0.00"), sa_column=Column(Numeric(10, 2)))
    line_total: Decimal = Field(sa_column=Column(Numeric(10, 2)))

    __table_args__ = (
        CheckConstraint("quantity > 0"),
        CheckConstraint("line_total = (quantity * unit_price) - discount_amount"),
    )


## 2. Core Application Actions (Python Logic)

Action 1/14: User Sign Up (Create New User)

In [2]:
# Python Implementation Snippet (from backend/app/handlers/users_handler.py)
def signup(data, session, pwd_context):
    # Validation checks for uniqueness
    existing_email = session.exec(select(User).where(User.email == data.email)).first()
    if existing_email: raise HTTPException(status.HTTP_400_BAD_REQUEST, detail="Email already registered")
    # ... manager secret check, etc. ...
    
    hashed = pwd_context.hash(data.password)

    # INSERT statement equivalent (creates the User record)
    user = User(
        email=str(data.email), 
        hashed_password=hashed, 
        username=data.username, 
        name=data.name, 
        role=data.role
    )
    session.add(user)
    session.commit()
    session.refresh(user)
    return user

Action 2/14: User Sign In (Authentication)

In [ ]:
def signin(data: UserLogin, session: Session) -> Token:
    statement = select(User).where(or_(User.email == data.identifier, User.username == data.identifier))
    user = session.exec(statement).first()
    if not user:
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid credentials")
    if not pwd_context.verify(data.password, user.hashed_password):
        raise HTTPException(status_code=status.HTTP_401_UNAUTHORIZED, detail="Invalid credentials")
    token = create_access_token(user_uid=user.uid)
    return Token(access_token=token)

Action 3/14: Create Product (Manager Action)

In [4]:
# Python Implementation Snippet (from backend/app/routes/products.py)
def create_product_action(data, session):
    # Check barcode uniqueness and price constraint (SELECT/Python logic)
    existing = session.exec(select(Product).where(Product.barcode == data.barcode)).first()
    if existing: raise HTTPException(400, detail="Barcode already exists")
    
    # Check selling price constraint before creating
    if data.selling_price < data.cost_price: raise HTTPException(status.HTTP_400_BAD_REQUEST, detail="Selling price cannot be less than cost price")
    
    # INSERT statement equivalent
    p = Product.model_validate(data)
    session.add(p)
    session.commit()
    session.refresh(p)
    return p

Action 4/14: Search Product by Query (POS/Inventory Lookup)

In [5]:
# Python Implementation Snippet (from backend/app/routes/products.py)
def list_products(q, session):
    if q:
        pattern = f"%{q}%"
        # SELECT with OR condition and ILIKE (case-insensitive LIKE)
        stmt = select(Product).where(
            or_(
                Product.name.ilike(pattern),
                Product.brand.ilike(pattern),
                Product.category.ilike(pattern),
                Product.barcode.ilike(pattern),
            )
        ).limit(50)
        return session.exec(stmt).all()
    return session.exec(select(Product).limit(50)).all()

Action 5/14: Update Product Stock/Details (Manager Action)

In [6]:
# Python Implementation Snippet (from backend/app/routes/products.py)
def update_product_details_action(product_id: int, data, session):
    p = session.exec(select(Product).where(Product.product_id == product_id)).first()
    if not p: raise HTTPException(404, detail="Product not found")

    data_to_update = data.model_dump(exclude_unset=True)
    
    # Apply updates from request data (UPDATE equivalent)
    for field, value in data_to_update.items():
        setattr(p, field, value)
    
    # Re-check selling price constraint after updates
    if p.selling_price < p.cost_price:
        raise HTTPException(status.HTTP_400_BAD_REQUEST, detail="Selling price cannot be less than cost price")

    session.add(p)
    session.commit()
    session.refresh(p)
    return p

Action 6/14: Delete Product (Manager Action)

In [7]:
# Python Implementation Snippet (from backend/app/routes/products.py)
def delete_product_action(product_id: int, session):
    p = session.exec(select(Product).where(Product.product_id == product_id)).first()
    if not p: raise HTTPException(404, detail="Product not found")
        
    # DELETE statement equivalent
    session.delete(p)
    session.commit()
    return {"ok": True}

Action 7/14: Create Promotion (Manager Action)

In [8]:
# Python Implementation Snippet (from backend/app/routes/promotions.py)
def create_promotion_action(data, session):
    # Business validation checks (Python logic)
    if data.end_date < data.start_date: raise HTTPException(status.HTTP_400_BAD_REQUEST, detail="End date must be after start date")

    # INSERT statement equivalent
    promo = Promotion.model_validate(data)
    session.add(promo)
    session.commit()
    session.refresh(promo)
    return promo

Action 8/14: Update Promotion (Manager Action)

In [9]:
# Python Implementation Snippet (from backend/app/routes/promotions.py)
def update_promotion_action(promotion_id: int, data, session):
    promo = session.exec(select(Promotion).where(Promotion.promotion_id == promotion_id)).first()
    if not promo: raise HTTPException(404, detail="Promotion not found")

    update_data = data.model_dump(exclude_unset=True)
    
    # Apply changes (UPDATE equivalent)
    for field, value in update_data.items():
        setattr(promo, field, value)

    # Re-validation checks after applying changes
    if promo.discount_type == 'PERCENTAGE' and promo.discount_value > Decimal('100.00'):
        raise HTTPException(status.HTTP_400_BAD_REQUEST, detail="Percentage discount cannot exceed 100.00")

    session.add(promo)
    session.commit()
    session.refresh(promo)
    return promo

Action 9/14: Delete Promotion (Unlink Products & Delete)

In [10]:
# Python Implementation Snippet (from backend/app/routes/promotions.py)
def delete_promotion_action(promotion_id: int, session):
    promo = session.exec(select(Promotion).where(Promotion.promotion_id == promotion_id)).first()
    if not promo: raise HTTPException(404, detail="Promotion not found")
        
    # Step 1: Unlink from products (SELECT + UPDATE equivalent)
    products_to_unlink = session.exec(select(Product).where(Product.promotion_id == promotion_id)).all()
    for product in products_to_unlink:
        product.promotion_id = None
        session.add(product)
        
    # Step 2: Delete the promotion (DELETE statement equivalent)
    session.delete(promo)
    session.commit()
    return {"ok": True}

Action 10/14: List Employees (Manager Report)

In [11]:
# Python Implementation Snippet (from backend/app/handlers/users_handler.py)
def list_employees_action(session, role: str | None = None):
    stmt = select(User)
    if role in ("manager", "cashier"):
        # SELECT with optional WHERE filter
        stmt = stmt.where(User.role == role)
        
    users = session.exec(stmt).all()
    # Application sort logic: manager first
    users.sort(key=lambda u: 0 if u.role == "manager" else 1)
    return users

Action 11/14: Identify Low Stock Products (Dashboard KPI)

In [12]:
# Python Implementation Snippet (derived from inventory/dashboard logic)
def get_low_stock_products(session):
    # SELECT with WHERE condition: stock < min_stock, ordered by deficiency
    stmt = select(Product).where(Product.stock_quantity < Product.min_stock).order_by(
        (Product.stock_quantity - Product.min_stock) 
    )
    return session.exec(stmt).all()

Action 12/14: Insert Transaction Record (Checkout Step 1)

In [13]:
# Python Implementation Snippet (from backend/app/routes/transactions.py)
def transaction_step_1_insert_tx(tx_data, session):
    # INSERT into transaction
    tx = Transaction(
        employee_id=tx_data.employee_id,
        member_id=tx_data.member_id,
        subtotal=tx_data.subtotal, 
        total_amount=tx_data.total_amount,
        payment_method=tx_data.payment_method,
        # ... other calculated fields like discounts ...
    )
    session.add(tx)
    session.commit()
    session.refresh(tx) # Crucial to retrieve the transaction_id
    return tx

Action 13/14: Insert Line Items & Update Inventory (Checkout Steps 2 & 3)

In [14]:
from typing import List

# Python Implementation Snippet (from backend/app/routes/transactions.py)
def transaction_step_2_items_and_stock_update(tx_id: int, items_to_save: List[TransactionItem], session):
    for item in items_to_save:
        # INSERT into transactionitem
        item.transaction_id = tx_id
        session.add(item)
        
        # UPDATE product stock (A11 Logic)
        prod = session.exec(select(Product).where(Product.product_id == item.product_id)).first()
        if prod:
            prod.stock_quantity -= item.quantity
            session.add(prod)
    # session.commit() is called once after this loop in the main route.
    pass

Action 14/14: Update Member Loyalty & Tier Check (Post-Transaction)

In [15]:
# Python Implementation Snippet (from backend/app/routes/transactions.py)
def transaction_step_3_update_member_loyalty(member, total_amount: Decimal, session):
    # Step 1: Update total spent/points (UPDATE equivalent)
    points_earned = int(total_amount.quantize(Decimal("1"), rounding=ROUND_HALF_UP))
    member.points_balance += points_earned
    member.total_spent += total_amount 
    session.add(member)

    # Step 2: Check and update tier (Python logic calling a helper function internally)
    # The update_member_tier function handles the SELECT/UPDATE for tier progression
    
    session.commit() # Commits all member changes